In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/College/Semester_5/ComputerVision/FINPRO/PCB_Defect_Detection/Dataset/
!ls

/content/drive/MyDrive/College/Semester_5/ComputerVision/FINPRO/PCB_Defect_Detection/Dataset
 archive.zip   dataset.zip  '[Processed]PCB_Data'


In [3]:
!unzip archive.zip -d /content/

Archive:  archive.zip
  inflating: /content/dataset/tobeaugmented/defected/20200707_085703.jpg  
  inflating: /content/dataset/tobeaugmented/defected/20200707_090641.jpg  
  inflating: /content/dataset/tobeaugmented/defected/20200707_091300.jpg  
  inflating: /content/dataset/tobeaugmented/defected/20200707_092157.jpg  
  inflating: /content/dataset/tobeaugmented/defected/20200707_092201.jpg  
  inflating: /content/dataset/tobeaugmented/defected/20200707_092731.jpg  
  inflating: /content/dataset/tobeaugmented/defected/20200707_093001.jpg  
  inflating: /content/dataset/tobeaugmented/defected/20200707_101023.jpg  
  inflating: /content/dataset/tobeaugmented/defected/20200707_101034.jpg  
  inflating: /content/dataset/tobeaugmented/defected/20200707_101039.jpg  
  inflating: /content/dataset/tobeaugmented/defected/20200707_101612.jpg  
  inflating: /content/dataset/tobeaugmented/defected/20200707_103238.jpg  
  inflating: /content/dataset/tobeaugmented/defected/20200707_103242.jpg  
  i

In [4]:
%cd /content/

/content


# K-Means

In [ ]:
# prompt: Reference from the code below, and add a feature extractor (SIFT or ORB) in the implementation use K-Means Clustering Algorithm.
# Before doing it, combine the dataset from defected and non-defected examples first then shuffle to feed into the algorithm

from google.colab import drive
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.utils import shuffle
import random

# Feature extraction function (using SIFT)
def extract_features(image):
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return keypoints, descriptors

def detect_defects(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not load image at {image_path}")
        return None, False

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    keypoints, descriptors = extract_features(gray)

    if descriptors is None:
        print(f"Warning: No keypoints detected in {image_path}")
        return img, False

    kmeans = KMeans(n_clusters=2, random_state=0) # Adjust n_clusters
    kmeans.fit(descriptors)
    labels = kmeans.labels_

    defect_image = img.copy()
    isDefect = False
    for i, keypoint in enumerate(keypoints):
        if labels[i] == 1: # Assumes cluster 1 represents defects
            x, y = int(keypoint.pt[0]), int(keypoint.pt[1])
            if 0 <= x < img.shape[1] and 0 <= y < img.shape[0]:
                cv2.circle(defect_image, (x, y), 5, (0, 0, 255), -1)
                isDefect = True
    return defect_image, isDefect

# Combine and shuffle datasets
defected_dir = "/content/dataset/tobeaugmented/defected"
non_defected_dir = "/content/dataset/tobeaugmented/non-defect" # Path to non-defected images
output_dir = "/content/defect_results_kmeans_combined/"
os.makedirs(output_dir, exist_ok=True)

defected_images = [os.path.join(defected_dir, f) for f in os.listdir(defected_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
non_defected_images = [os.path.join(non_defected_dir, f) for f in os.listdir(non_defected_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]

combined_images = defected_images + non_defected_images
random.shuffle(combined_images)

for image_path in combined_images:
    defect_img, isDefect = detect_defects(image_path)
    if defect_img is not None:
        filename = os.path.basename(image_path)
        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, defect_img)